In [ ]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video
import nb_helpers
import pandas as pd
import glob
import numpy.linalg as LA
import os
import math

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
poselandmarks_list = mp.solutions.pose.PoseLandmark

###### In the below code we have different functions thare are being used for the following:<br>
1) outliers_new function is being used to remove outliers from the x, y, z coordinate files for the exercises. The emthod for removing outliers is the simple IQR method. any values greater than Q3 + 1.5 * IQR and less than Q1 - 1.5 * IQR are treated as an outlier.<br>
2)The moving function is used to detect the moving parts. Mean of the median absolute deviation is used to check if a joint is moving or not. The threshold used is 0.08. If any of the x, y or z coordinates of the respective joints showing greater than 0.08 deviation, then that joint is treated as a moving part. This function was used earlier in the research but then dropped because the detection of the moving and non moving parts did not end up being so good.<br>
3) The moving_distances function is used to calculate the distance between two joints as a way to calculate the amount of movement of each of the joints. For example, if we want to check degree of movement of hip, then we calculate the distance betwwen the shoulder joint and the knee joint. The Q1 and Q3 values are used instead od the max and the min value in this function.<br>
4) The moving_distances 2 function is also for the same function as the moving_distances function but it uses the min and the max values instead of the uqrtiles to measure the range of the meovement of the joints. Also, we are calcultaing the mean of the median absolute deviation for each of the joints which I thought of used as weights for each of the joint movements.<br>
5) The calc_ distances is being used to calculate the distance changes at each of the frames in the videos. The method of calculating the distance changes is the same as described in point 3 above.<br>
6) The angle_calculator and calc_angles are used together to calculate the angle changes at the joints.<br>
7) The proximity function is used to check if the camera is front facing left side facing or right side facing based on the z coordinates of the left hip and the right hip.
8) The standing_pose function is used to detect the bodt position. In this function I am using the x, y, z coordinates deviation and using a threshold to detect the body position based on the amount of movement of the joints. Any movement above the threshold is treated as that joint moving and then this information is used to detect the body position.

In [ ]:
# Outlier Removal
def outliers_new_3(df_xyz):
    outlier_list = []

    for i in range(0, df_xyz.shape[1], 3):
        for j in range(i, i + 3):
            qq1 = np.quantile(df_xyz.iloc[:, j], 0.25)
            qq3 = np.quantile(df_xyz.iloc[:, j], 0.75)
            iqr2 = qq3 - qq1
            upp = qq3 + (1.5 * iqr2)
            low = qq1 - (1.5 * iqr2)
            check_upp = df_xyz.iloc[:, j] > upp
            check_low = df_xyz.iloc[:, j] < low
            outlier_list = outlier_list + [r for r in range(0, len(check_upp)) if check_upp[r]]
            outlier_list = outlier_list + [r for r in range(0, len(check_low)) if check_low[r]]
    outlier_list = np.unique(outlier_list)
    df_xyz_2 = df_xyz.drop(outlier_list, axis=0)

    return df_xyz_2

# Moving Parts
def moving(df_4, index_exercise):
    for i_3 in range(0,df_4.shape[1],3):
        count = 0
        for j_3 in range(i_3, i_3+3):
            a_2 = df_4.iloc[:,j_3].tolist()
            a_2 = [x for x in a_2 if x != 99]
            median = np.quantile(a_2, 0.5)
            deviation = np.mean([abs(x - median) for x in a_2])
            if deviation > 0.08:
                count = count + 1
        if count > 0:
            df_5.iloc[index_exercise,int((i_3+3)/3)] = "Moving"
        else:
            df_5.iloc[index_exercise,int((i_3+3)/3)] = "Not Moving"
            
def moving_distances(df_distances, index_exercise):
    for i in range(1,9):
        q1 = np.quantile(df_distances.iloc[:,i],0.25)
        q3 = np.quantile(df_distances.iloc[:,i],0.75)
        iqr = q3 - q1
        df_6.iloc[index_exercise,i] = iqr

def moving_distances_2(df_distances, index_exercise):
    for i in range(1,9):
        df_7.iloc[index_exercise,i] = max(df_distances.iloc[:,i]) - min(df_distances.iloc[:,i])
        q2 = np.quantile(df_distances.iloc[:,i],0.5)
        df_7.iloc[index_exercise,i+8] = np.mean(abs(df_distances.iloc[:,i] - q2))

def calc_distances(xyz_new, name):
    column_names = ['Exercise', 'Left Shoulder Angle','Left Elbow Angle','Left Hip Angle',
                    'Left Knee Angle','Right Shoulder Angle','Right Elbow Angle','Right Hip Angle','Right Knee Angle',
                    'Left Shoulder Hip', 'Left Hip Knee', 'Left Knee Ankle', 'Right Shoulder Hip', 'Right Hip Knee',
                    'Right Knee Ankle']
    exercise = pd.DataFrame(index=range(xyz_new.shape[0]), columns=column_names)

    # Angle Calculation
    for i_2 in range(xyz_new.shape[0]):

        exercise.iloc[i_2, 0] = name

        # Left Side Parts

        # Left Shoulder
        exercise.iloc[i_2, 1] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 7] - xyz_new.iloc[i_2, 19], xyz_new.iloc[i_2, 8] - xyz_new.iloc[i_2, 20],
             xyz_new.iloc[i_2, 9] - xyz_new.iloc[i_2, 21]]))

        # Left Elbow
        exercise.iloc[i_2, 2] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 1] - xyz_new.iloc[i_2, 13], xyz_new.iloc[i_2, 2] - xyz_new.iloc[i_2, 14],
             xyz_new.iloc[i_2, 3] - xyz_new.iloc[i_2, 15]]))

        # Left Hip
        exercise.iloc[i_2, 3] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 1] - xyz_new.iloc[i_2, 25], xyz_new.iloc[i_2, 2] - xyz_new.iloc[i_2, 26],
             xyz_new.iloc[i_2, 3] - xyz_new.iloc[i_2, 27]]))

        # Left Knee
        exercise.iloc[i_2, 4] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 19] - xyz_new.iloc[i_2, 31], xyz_new.iloc[i_2, 20] - xyz_new.iloc[i_2, 32],
             xyz_new.iloc[i_2, 21] - xyz_new.iloc[i_2, 33]]))
        
        # Left Shoulder Hip
        exercise.iloc[i_2, 9] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 1] - xyz_new.iloc[i_2, 19], xyz_new.iloc[i_2, 2] - xyz_new.iloc[i_2, 20],
             xyz_new.iloc[i_2, 3] - xyz_new.iloc[i_2, 21]]))
        
        # Left Hip Knee
        exercise.iloc[i_2, 10] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 25] - xyz_new.iloc[i_2, 19], xyz_new.iloc[i_2, 26] - xyz_new.iloc[i_2, 20],
             xyz_new.iloc[i_2, 27] - xyz_new.iloc[i_2, 21]]))
        
        # Left Knee Ankle
        exercise.iloc[i_2, 11] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 25] - xyz_new.iloc[i_2, 31], xyz_new.iloc[i_2, 26] - xyz_new.iloc[i_2, 32],
             xyz_new.iloc[i_2, 27] - xyz_new.iloc[i_2, 33]]))

        # Right Side Parts

        # Right Shoulder
        exercise.iloc[i_2, 5] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 10] - xyz_new.iloc[i_2, 22], xyz_new.iloc[i_2, 11] - xyz_new.iloc[i_2, 23],
             xyz_new.iloc[i_2, 12] - xyz_new.iloc[i_2, 24]]))

        # Right Elbow
        exercise.iloc[i_2, 6] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 4] - xyz_new.iloc[i_2, 16], xyz_new.iloc[i_2, 5] - xyz_new.iloc[i_2, 17],
             xyz_new.iloc[i_2, 6] - xyz_new.iloc[i_2, 18]]))

        # Right Hip
        exercise.iloc[i_2, 7] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 4] - xyz_new.iloc[i_2, 28], xyz_new.iloc[i_2, 5] - xyz_new.iloc[i_2, 29],
             xyz_new.iloc[i_2, 6] - xyz_new.iloc[i_2, 30]]))

        # Right Knee
        exercise.iloc[i_2, 8] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 22] - xyz_new.iloc[i_2, 34], xyz_new.iloc[i_2, 23] - xyz_new.iloc[i_2, 35],
             xyz_new.iloc[i_2, 24] - xyz_new.iloc[i_2, 36]]))
        
        # Right Shoulder Hip
        exercise.iloc[i_2, 12] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 4] - xyz_new.iloc[i_2, 22], xyz_new.iloc[i_2, 5] - xyz_new.iloc[i_2, 23],
             xyz_new.iloc[i_2, 6] - xyz_new.iloc[i_2, 24]]))
        
        # Right Hip Knee
        exercise.iloc[i_2, 13] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 28] - xyz_new.iloc[i_2, 22], xyz_new.iloc[i_2, 29] - xyz_new.iloc[i_2, 23],
             xyz_new.iloc[i_2, 30] - xyz_new.iloc[i_2, 24]]))
        
        # Right Knee Ankle
        exercise.iloc[i_2, 14] = np.linalg.norm(np.array([xyz_new.iloc[i_2, 28] - xyz_new.iloc[i_2, 34], xyz_new.iloc[i_2, 29] - xyz_new.iloc[i_2, 35],
             xyz_new.iloc[i_2, 30] - xyz_new.iloc[i_2, 36]]))

    exercise.to_csv("Distances/" + name + ".csv", index=False)
    
    return exercise

def angle_calculator(e, f):
    inner = np.inner(e, f)
    norms = LA.norm(e) * LA.norm(f)
    cos = inner / norms
    rad = np.arccos(np.clip(cos, -1.0, 1.0))
    deg = np.rad2deg(rad)
    return deg

def calc_angles (xyz_new, name):
    column_names = ['Exercise', 'Left Shoulder Angle','Left Elbow Angle','Left Hip Angle','Left Knee Angle','Right Shoulder Angle','Right Elbow Angle','Right Hip Angle','Right Knee Angle']
    exercise = pd.DataFrame(index=range(xyz_new.shape[0]), columns=column_names)

    # Angle Calculation
    for i_2 in range(xyz_new.shape[0]):
        
        # Left Side Parts
        l_elb_l_shol = np.array(
            [xyz_new.iloc[i_2, 1] - xyz_new.iloc[i_2, 7], xyz_new.iloc[i_2, 2] - xyz_new.iloc[i_2, 8],
             xyz_new.iloc[i_2, 3] - xyz_new.iloc[i_2, 9]])  # for elbow angle
        l_elb_l_wr = np.array(
            [xyz_new.iloc[i_2, 13] - xyz_new.iloc[i_2, 7], xyz_new.iloc[i_2, 14] - xyz_new.iloc[i_2, 8],
             xyz_new.iloc[i_2, 15] - xyz_new.iloc[i_2, 9]])  # for elbow angle
        l_shol_l_elb = np.array(
            [xyz_new.iloc[i_2, 7] - xyz_new.iloc[i_2, 1], xyz_new.iloc[i_2, 8] - xyz_new.iloc[i_2, 2],
             xyz_new.iloc[i_2, 9] - xyz_new.iloc[i_2, 3]])  # for shoulder angle
        l_shol_l_hip = np.array(
            [xyz_new.iloc[i_2, 19] - xyz_new.iloc[i_2, 1], xyz_new.iloc[i_2, 20] - xyz_new.iloc[i_2, 2],
             xyz_new.iloc[i_2, 21] - xyz_new.iloc[i_2, 3]])  # for shoulder angle
        l_hip_l_knee = np.array(
            [xyz_new.iloc[i_2, 25] - xyz_new.iloc[i_2, 19], xyz_new.iloc[i_2, 26] - xyz_new.iloc[i_2, 20],
             xyz_new.iloc[i_2, 27] - xyz_new.iloc[i_2, 21]])  # for hip angle
        l_hip_l_shol = np.array(
            [xyz_new.iloc[i_2, 1] - xyz_new.iloc[i_2, 19], xyz_new.iloc[i_2, 2] - xyz_new.iloc[i_2, 20],
             xyz_new.iloc[i_2, 3] - xyz_new.iloc[i_2, 21]])  # for hip angle
        l_knee_l_ankle = np.array(
            [xyz_new.iloc[i_2, 31] - xyz_new.iloc[i_2, 25], xyz_new.iloc[i_2, 32] - xyz_new.iloc[i_2, 26],
             xyz_new.iloc[i_2, 33] - xyz_new.iloc[i_2, 27]])  # for knee angle
        l_knee_l_hip = np.array(
            [xyz_new.iloc[i_2, 19] - xyz_new.iloc[i_2, 25], xyz_new.iloc[i_2, 20] - xyz_new.iloc[i_2, 26],
             xyz_new.iloc[i_2, 21] - xyz_new.iloc[i_2, 27]])  # for knee angle

        # Right Side Parts
        r_elb_r_shol = np.array(
            [xyz_new.iloc[i_2, 4] - xyz_new.iloc[i_2, 10], xyz_new.iloc[i_2, 5] - xyz_new.iloc[i_2, 11],
             xyz_new.iloc[i_2, 6] - xyz_new.iloc[i_2, 12]])  # for elbow angle
        r_elb_r_wr = np.array(
            [xyz_new.iloc[i_2, 16] - xyz_new.iloc[i_2, 10], xyz_new.iloc[i_2, 17] - xyz_new.iloc[i_2, 11],
             xyz_new.iloc[i_2, 18] - xyz_new.iloc[i_2, 12]])  # for elbow angle
        r_shol_r_elb = np.array(
            [xyz_new.iloc[i_2, 10] - xyz_new.iloc[i_2, 4], xyz_new.iloc[i_2, 11] - xyz_new.iloc[i_2, 5],
             xyz_new.iloc[i_2, 12] - xyz_new.iloc[i_2, 6]])  # for shoulder angle
        r_shol_r_hip = np.array(
            [xyz_new.iloc[i_2, 22] - xyz_new.iloc[i_2, 4], xyz_new.iloc[i_2, 23] - xyz_new.iloc[i_2, 5],
             xyz_new.iloc[i_2, 24] - xyz_new.iloc[i_2, 6]])  # for shoulder angle
        r_hip_r_knee = np.array(
            [xyz_new.iloc[i_2, 28] - xyz_new.iloc[i_2, 22], xyz_new.iloc[i_2, 29] - xyz_new.iloc[i_2, 23],
             xyz_new.iloc[i_2, 30] - xyz_new.iloc[i_2, 24]])  # for hip angle
        r_hip_r_shol = np.array(
            [xyz_new.iloc[i_2, 4] - xyz_new.iloc[i_2, 22], xyz_new.iloc[i_2, 5] - xyz_new.iloc[i_2, 23],
             xyz_new.iloc[i_2, 6] - xyz_new.iloc[i_2, 24]])  # for hip angle
        r_knee_r_ankle = np.array(
            [xyz_new.iloc[i_2, 34] - xyz_new.iloc[i_2, 28], xyz_new.iloc[i_2, 35] - xyz_new.iloc[i_2, 29],
             xyz_new.iloc[i_2, 36] - xyz_new.iloc[i_2, 30]])  # for knee angle
        r_knee_r_hip = np.array(
            [xyz_new.iloc[i_2, 22] - xyz_new.iloc[i_2, 28], xyz_new.iloc[i_2, 23] - xyz_new.iloc[i_2, 29],
             xyz_new.iloc[i_2, 24] - xyz_new.iloc[i_2, 30]])  # for knee angle

        exercise.iloc[i_2, 0] = name

        # Left Elbow Angle
        exercise.iloc[i_2, 2] = angle_calculator(l_elb_l_shol, l_elb_l_wr)

        # Left Shoulder Angle
        exercise.iloc[i_2, 1] = angle_calculator(l_shol_l_elb, l_shol_l_hip)
        # print("Left Shoulder Angle")
        # print(cos_l_shol, rad_l_shol, deg_l_shol)

        # Left Hip
        exercise.iloc[i_2, 3] = angle_calculator(l_hip_l_knee, l_hip_l_shol)
        # print("Left Hip Angle")
        # print(cos_l_hip, rad_l_hip, deg_l_hip)

        # Left Knee
        exercise.iloc[i_2, 4] = angle_calculator(l_knee_l_ankle, l_knee_l_hip)
        # print("Left Knee Angle")
        # print(cos_l_knee, rad_l_knee, deg_l_knee)

        # Right Angles Calculation

        # Right Elbow Angle
        exercise.iloc[i_2, 6] = angle_calculator(r_elb_r_shol, r_elb_r_wr)
        # print("Right Elbow Angle")
        # print(cos_r_elb, rad_r_elb, deg_r_elb)

        # Right Shoulder Angle
        exercise.iloc[i_2, 5] = angle_calculator(r_shol_r_elb, r_shol_r_hip)
        # print("Right Shoulder Angle")
        # print(cos_r_shol, rad_r_shol, deg_r_shol)

        # Right Hip
        exercise.iloc[i_2, 7] = angle_calculator(r_hip_r_knee, r_hip_r_shol)
        # print("Right Hip Angle")
        # print(cos_r_hip, rad_r_hip, deg_r_hip)

        # Right Knee
        exercise.iloc[i_2, 8] = angle_calculator(r_knee_r_ankle, r_knee_r_hip)
        # print("Right Knee Angle")
        # print(cos_r_knee, rad_r_knee, deg_r_knee)

    exercise.to_csv("Angles/" + name + ".csv", index=False)
    
    return exercise

def proximity (df,index_exercise):
    a1 = np.mean(df.iloc[:,21])
    a2 = np.mean(df.iloc[:,24])
    diff = abs(a1 - a2)
    if diff > 0.05:
        if a1 < a2:
            df_6.iloc[index_exercise,9] = 1 # left side facing camera
            df_7.iloc[index_exercise,17] = 1
            return 1
        else:
            df_6.iloc[index_exercise,9] = 2 # right side facing camera
            df_7.iloc[index_exercise,17] = 2
            return 2
    else:
        df_6.iloc[index_exercise,9] = 3 # front facing camera
        df_7.iloc[index_exercise,17] = 3
        return 3
        
def standing_pose (index_exercise, camera_angle, dist, ang):
    if camera_angle == 2:
        # Standing Pose Calculation
        print(np.mean(abs(ang.iloc[:,7] - np.median(ang.iloc[:,7]))), np.mean(abs(ang.iloc[:,8] - np.median(ang.iloc[:,8]))))
        a = np.std(dist.iloc[:,7])
        b = np.std(dist.iloc[:,8])
        print(a, b)
        if a < 0.03 and b < 0.03:
            print("Inside " + str(camera_angle))
            c = np.mean(dist.iloc[:,7])
            d = np.mean(dist.iloc[:,12]) + np.mean(dist.iloc[:,13])
            e = np.mean(dist.iloc[:,8])
            f = np.mean(dist.iloc[:,13]) + np.mean(dist.iloc[:,14])
            if (c / d) > 0.85 and (e / f) > 0.85:
                df_9.iloc[index_exercise, 1] = 1
            else:
                df_9.iloc[index_exercise, 1] = 0
            
            # Sitting Pose Calculation
            d_2 = (np.mean(dist.iloc[:,12])**2 + np.mean(dist.iloc[:,13])**2)**0.5
            f_2 = (np.mean(dist.iloc[:,13])**2 + np.mean(dist.iloc[:,14])**2)**0.5
            if df_9.iloc[index_exercise, 1] == 0:
                if c < 1.15 * d_2 and e < 1.15 * f_2:
                    df_9.iloc[index_exercise, 2] = 1
                else:
                    df_9.iloc[index_exercise, 2] = 0
            else:
                df_9.iloc[index_exercise, 2] = 0
        else:
            df_9.iloc[index_exercise, [1, 2, 3]] = 0

    else:
        # Standing Pose Calculation
        print(np.mean(abs(ang.iloc[:,1] - np.median(ang.iloc[:,1]))), np.mean(abs(ang.iloc[:,2] - np.median(ang.iloc[:,2]))),
            np.mean(abs(ang.iloc[:,3] - np.median(ang.iloc[:,3]))), np.mean(abs(ang.iloc[:,4] - np.median(ang.iloc[:,4]))))
        a = np.std(dist.iloc[:,3])
        b = np.std(dist.iloc[:,4])
        print(a, b)
        if a < 0.03 and b < 0.03:
            print("Inside " + str(camera_angle))
            c = np.mean(dist.iloc[:,3])
            d = np.mean(dist.iloc[:,9]) + np.mean(dist.iloc[:,10])
            e = np.mean(dist.iloc[:,4])
            f = np.mean(dist.iloc[:,10]) + np.mean(dist.iloc[:,11])
            if (c / d) > 0.85 and (e / f) > 0.85:
                df_9.iloc[index_exercise, 1] = 1
            else:
                df_9.iloc[index_exercise, 1] = 0

            # Sitting Pose Calculation
            d_2 = (np.mean(dist.iloc[:,9])**2 + np.mean(dist.iloc[:,10])**2)**0.5
            f_2 = (np.mean(dist.iloc[:,10])**2 + np.mean(dist.iloc[:,11])**2)**0.5
            print(c,d_2,e,f_2)
            print(c / d_2, e / f_2)
            if df_9.iloc[index_exercise, 1] == 0:
                if c < 1.15 * d_2 and e < 1.15 * f_2:
                    df_9.iloc[index_exercise, 2] = 1
                else:
                    df_9.iloc[index_exercise, 2] = 0
            else:
                df_9.iloc[index_exercise, 2] = 0
        else:
            df_9.iloc[index_exercise, [1, 2, 3]] = 0

###### All of the functions mentioned above are used below for each of the exercises to get the required features.

In [ ]:
exercise_names = pd.read_csv("Exercise Names.csv")
column_names_parts = ['Exercise', 'Left Shoulder Angle','Left Elbow Angle','Left Hip Angle','Left Knee Angle',
                      'Right Shoulder Angle','Right Elbow Angle','Right Hip Angle','Right Knee Angle',
                      'Facing Side']
column_names_2 = ['Exercise', 'Left Shoulder Angle','Left Elbow Angle','Left Hip Angle','Left Knee Angle',
                  'Right Shoulder Angle','Right Elbow Angle','Right Hip Angle','Right Knee Angle',
                  'Left Shoulder Weight','Left Elbow Weight','Left Hip Weight','Left Knee Weight', 
                  'Right Shoulder Weight','Right Elbow Weight','Right Hip Weight','Right Knee Weight','Facing Side']
column_names_3 = ['Exercise', 'Standing', 'Sitting', 'Lying']
df_6 = pd.DataFrame(index=range(exercise_names.shape[0]), columns=column_names_parts)
df_7 = pd.DataFrame(index=range(exercise_names.shape[0]), columns=column_names_2)
df_9 = pd.DataFrame(index=range(exercise_names.shape[0]), columns=column_names_3)
df_9.iloc[:,0] = exercise_names.iloc[:,0]
# exercise_names.shape[0]

for index in range(exercise_names.shape[0]):
    name = exercise_names.iloc[index,0] 
    
    file = "C:\\Users\\muham\\Desktop\\Edited Exercise Videos\\" + name + ".mp4"
    if not os.path.exists(name + " " + str('frames')):
        os.makedirs(name + " " + str('frames'))
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        # Create VideoCapture object
        cap = cv2.VideoCapture(file)

        # Raise error if file cannot be opened
        if cap.isOpened() == False:
            print("Error opening video stream or file")
            raise TypeError

        # Get the number of frames in the video
        length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Create a NumPy array to store the pose data as before
        # The shape is 3x33x144 - 3D XYZ data for 33 landmarks across 144 frames
        data = np.empty((4, len(poselandmarks_list), length))    

        # For each image in the video, extract the spatial pose data and save it in the appropriate spot in the `data` array 
        frame_num = 0
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

#             annotated_image = image.copy()
#             # Draw pose landmarks on the image.
#             mp_drawing.draw_landmarks(
#                 annotated_image,
#                 results.pose_landmarks,
#                 mp_pose.POSE_CONNECTIONS,
#                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
#             cv2.imwrite(name + ' frames/' + str(frame_num) + '.png', annotated_image)

            if hasattr(results.pose_landmarks, 'landmark'):
                landmarks = results.pose_landmarks.landmark
                for i in range(len(mp_pose.PoseLandmark)):
                    data[:, i, frame_num] = (landmarks[i].x, landmarks[i].y, landmarks[i].z, landmarks[i].visibility)

            frame_num += 1

        # Close the video file
        cap.release()

    df = pd.DataFrame(index=range(data.shape[2]), columns=range(48))

    for i_2 in range(data.shape[2]):
        k = 0
        for i_3 in [11,12,13,14,15,16,23,24,25,26,27,28]:
            for i_4 in range(data.shape[0]):
                df.iloc[i_2, k] = data[i_4, i_3, i_2]
                k = k + 1

    df.to_csv("With Visibility/" + name + " with visibility.csv", index=False)
    print(name)
    print(df.shape)

    df = df.drop([3,7,11,15,19,23,27,31,35,39,43,47], axis=1)
    df_new = df.copy(deep=True)

    df_wo_outlier = outliers_new_3(df_new)
    df_wo_outlier.to_csv("Without Outlier/" + name + ".csv", index=False)
    print(df_wo_outlier.shape)
    
    xyz_new = df_wo_outlier.copy(deep=True)
    a = [name] * xyz_new.shape[0]
    xyz_new.insert(0, "Exercise", a)
    
    dist = calc_distances(xyz_new, name)
    ang = calc_angles(xyz_new, name)

    df_6.iloc[index,0] = name
    df_7.iloc[index,0] = name
    
    c_angle = proximity(xyz_new, index)  # Camera Angle return value
    
    standing_pose(index, c_angle, dist, ang)
    
    moving_distances(dist, index)  # Inter quartile range
    moving_distances_2(dist,index)  # Range and Median Absolute Deviation

# df_6.to_csv("Distance Deviation.csv", index=False)
# df_7.to_csv("Distance Deviation 2.csv", index=False)
df_9.to_csv("Body Position.csv", index=False)  # Body position such as Standing, Sitting, etc